# make dataset from crawled advertisements

In [1]:
!gdown --fuzzy https://drive.google.com/file/d/1uPhn_pg_-aJNiAxm60Rpwb0MbzxzGNQ6/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=1uPhn_pg_-aJNiAxm60Rpwb0MbzxzGNQ6
To: /content/mlops.zip
100% 11.9M/11.9M [00:00<00:00, 17.7MB/s]


In [2]:
!unzip mlops.zip

Archive:  mlops.zip
  inflating: outputfile7.json        
  inflating: outputfile              
  inflating: outputfile.json         
  inflating: outputfile2.json        
  inflating: outputfile4.json        
  inflating: outputfile5.json        
  inflating: outputfile6.json        


In [1]:
import json
import pandas as pd
import numpy as np
from keras.utils import pad_sequences
import torch
import torchvision
from google.colab import drive

In [2]:
data = []
f = open('outputfile.json')
data += json.load(f)
for i in range(2 , 8):
    if i!=3:
        f = open(f'outputfile{i}.json')
        data+= json.load(f)

In [3]:
listKala = []
for i in data:
    if i['category']!= [] :
        if 'mainTitle' in i.keys():
            if type(i['mainTitle']) == str:
    # and type(i['mainTitle']) =='str':
                listKala.append(i)
len(listKala)

32728

In [4]:
df = pd.DataFrame(listKala)
col = df.columns
colum1 = df.columns
listdrop = ['نوع آگهی' , 'آخرین به‌روزرسانی آگهی' , 'نوع کالا' , 'شیوهٔ پرداخت' , 'آگهی‌دهنده' , 'وضعیت' , 'link']
df.drop(listdrop, inplace=True, axis=1)
len(df.columns)

89

In [5]:
cat0 = [cat[0] for cat in list(df['category'])] 
cat1 = [cat[1] for cat in list(df['category'])]
cat2 = []
for cat in list(df['category']):
    if len(cat) < 3:
        cat2.append("not")
    else:
        cat2.append(cat[2])
df.insert(loc=1, column="cat0", value=cat0)
df.insert(loc=1, column="cat1", value=cat1)
df.insert(loc=1, column="cat2", value=cat2) 
keys = df.columns

# make second description from other feautures

In [6]:
notMaking = ['وضعیت' , 'description' , 'secondTitle' , 'category' , 'قیمت'  , 'link' , 'mainTitle' , 'cat0' , 'cat1' , 'cat2'  ]
listOfDescriptions = []
for idx , row in df.iterrows():
    listOfSentences = []  
    for key in keys:
        if key not in notMaking and type(row[key])== str :
            sentence = row['mainTitle'] + " ،"+ key + " " + str(row[key])+" است. " 
            listOfSentences.append(sentence)
    listOfSentences.append('این کالا ' + row['mainTitle'] + ' است ')
    madeSentence = ''
    for sentence in listOfSentences:
        madeSentence = madeSentence + sentence + ' '
    listOfDescriptions.append(madeSentence)
df.insert(loc=1, column="description2", value=listOfDescriptions)

In [ ]:
df.to_csv('advertisementData.csv')

# change most of features to zeroOne

In [7]:
notMakingZeroAndOne = [ 'description2' , 'قیمت' , 'category' , 'secondTitle' , 'description' , 'mainTitle'  , 'cat0' , 'cat1' , 'cat2' ]

In [8]:
df2ZeroAndOne= df.copy()

In [9]:
for column in df2ZeroAndOne.columns:
    if column not in notMakingZeroAndOne:
        df2ZeroAndOne[column] = df2ZeroAndOne[column].replace(np.nan, 0)
        # df2ZeroAndOne[column] = np.where(df2ZeroAndOne[column]== 0 , 0, 1)


In [10]:
for column in df2ZeroAndOne.columns:
    if column not in notMakingZeroAndOne:
        # df2ZeroAndOne[column] = df2ZeroAndOne[column].replace(np.nan, 0)
        df2ZeroAndOne[column] = np.where(df2ZeroAndOne[column]== 0 , 0, 1)

In [11]:
listZeroOne = []
for col in df2ZeroAndOne.columns:
    if col not in notMakingZeroAndOne:
        listZeroOne.append(col)


dfZeroOne = df2ZeroAndOne[listZeroOne]

In [12]:
descriptions2 = list(df2ZeroAndOne['description2'])
descriptions = list(df2ZeroAndOne['description'])
title = list(df2ZeroAndOne['mainTitle'])

In [13]:
dfZeroOne.to_csv('dataJustZeroOne.csv')

In [14]:
df2ZeroAndOne.to_csv('dataWithZeroOne.csv')

# take cls embedding from bert for main description and made description

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from transformers import AutoConfig, AutoTokenizer, AutoModel

config = AutoConfig.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
model = AutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased" )

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
input_tokens = []
for i in range(len(descriptions2)):
  input_tokens.append(tokenizer.encode(descriptions2[i], add_special_tokens=True))

In [18]:
len(input_tokens)
temp = []
for i in input_tokens:
  #print(len(i))
  temp.append(len(i))
np.max(temp) 

330

In [19]:
input_ids = pad_sequences(input_tokens, maxlen=340, dtype="long", value=0, truncating="post", padding="post")

In [20]:
def create_attention_mask(input_id):
  attention_masks = []
  for sent in input_ids:
    att_mask = [int(token_id > 0) for token_id in sent]  # create a list of 0 and 1.
    attention_masks.append(att_mask)  # basically attention_masks is a list of list
  return attention_masks

In [21]:
input_masks = create_attention_mask(input_ids)

In [26]:

input_ids = torch.tensor(input_ids).to('cuda:0')
attention_mask = torch.tensor(input_masks).to('cuda:0')

In [ ]:
listFeatures = []
listMainFeatures = []
model = model.to('cuda:0')
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask = attention_mask)
    listFeatures.append(last_hidden_states[0][:,0,:])


In [ ]:
torch.save(listFeatures , 'listMain.pt')